In [1]:
# !wget https://huggingface.co/datasets/theblackcat102/evol-codealpaca-v1/resolve/main/train.jsonl -O evol-codealpaca-v1.jsonl

In [2]:
import pandas as pd
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai7-aiservices-435901645.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

engine = 'gpt-4'

In [4]:
instructions = []
with open('evol-codealpaca-v1.jsonl') as fopen:
    for l in fopen:
        instructions.append(json.loads(l)['instruction'])
len(instructions)

111272

In [5]:
def predict(prompt, max_tokens = 1024):
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
       
    ]
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=max_tokens,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [6]:
prompt = f'terjemah ke bahasa melayu: {instructions[0]}'

In [7]:
!mkdir evol-codealpaca-v1-chatgpt4
# !rm -rf evol-codealpaca-v1/*

mkdir: cannot create directory ‘evol-codealpaca-v1-chatgpt4’: File exists


In [8]:
def generate(instruction, index, show = False):
    filename = f'evol-codealpaca-v1-chatgpt4/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        prompt = f'terjemah ke bahasa melayu: {instruction}'
        ins = predict(prompt)
        if show:
            print(ins)
        answer = predict(ins + ', jawab dalam bahasa melayu', max_tokens = 2048)
        d = {"instruction":ins,"output":answer}
        with open(filename, 'w') as fopen:
            json.dump(d, fopen)
    except Exception as e:
        print(e)
        pass

In [ ]:
from tqdm import tqdm

max_worker = 1
for i in tqdm(range((len(instructions) // 6) * 0, (len(instructions) // 6) * 1, max_worker)):
    b = instructions[i: i + max_worker]
    b = [(ins, i + no) for no, ins in enumerate(b)]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f[0], f[1]): f for f in b}

        for future in as_completed(futures):
            future.result()

  6%|██████▏                                                                                              | 1140/18545 [00:00<00:01, 11393.98it/s]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
unsupported operand type(s) for +: 'NoneType' and 'str'


 12%|████████████▋                                                                                          | 2280/18545 [00:14<02:00, 134.93it/s]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 17%|████████████████▉                                                                                 | 3206/18545 [3:02:13<173:27:50, 40.71s/it]

'content'
unsupported operand type(s) for +: 'NoneType' and 'str'


 20%|███████████████████▋                                                                             | 3758/18545 [9:11:30<811:32:10, 197.57s/it]

Request timed out: HTTPSConnectionPool(host='husein-ai7-aiservices-435901645.openai.azure.com', port=443): Read timed out. (read timeout=600)


 30%|█████████████████████████████                                                                     | 5510/18545 [24:19:19<88:01:54, 24.31s/it]